# Integração CoreML do RusTorch - Bindings Python

Este notebook demonstra como usar a funcionalidade CoreML do RusTorch através de bindings Python.

## Configuração e Importações

In [ ]:
# Importar bindings Python do RusTorch
try:
    import rustorch
    print(f"✅ Versão do RusTorch: {rustorch.__version__}")
    print(f"📝 Descrição: {rustorch.__description__}")
    print(f"👥 Autor: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Falha ao importar RusTorch: {e}")
    print("Por favor, compile com maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Plataforma: {platform.system()} {platform.release()}")
print(f"🐍 Versão do Python: {platform.python_version()}")

## Verificar Disponibilidade do CoreML

In [ ]:
# Verificar funcionalidade CoreML
try:
    # Verificar se CoreML está disponível
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML disponível: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML está disponível!")
        
        # Obter informações do dispositivo
        device_info = rustorch.get_coreml_device_info()
        print("📱 Informações do dispositivo CoreML:")
        print(device_info)
    else:
        print("⚠️ CoreML não está disponível")
        if platform.system() != "Darwin":
            print("CoreML está disponível apenas no macOS")
        else:
            print("Recursos CoreML podem não estar habilitados")
            
except AttributeError:
    print("❌ Funções CoreML não encontradas")
    print("Pode não estar compilado com recursos CoreML")
    coreml_available = False
except Exception as e:
    print(f"❌ Erro ao verificar CoreML: {e}")
    coreml_available = False

## Criação de Dispositivo CoreML e Operações

In [ ]:
if coreml_available:
    try:
        # Criar dispositivo CoreML
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ Dispositivo CoreML criado: {device}")
        
        # Obter informações do dispositivo
        print(f"🆔 ID do dispositivo: {device.device_id()}")
        print(f"✅ Disponível: {device.is_available()}")
        print(f"💾 Limite de memória: {device.memory_limit()} bytes")
        print(f"🧮 Limite de unidades de computação: {device.compute_units_limit()}")
        print(f"📚 Tamanho do cache do modelo: {device.model_cache_size()}")
        
        # Limpeza do cache
        device.cleanup_cache()
        print("🧹 Cache limpo")
        
    except Exception as e:
        print(f"❌ Erro na operação do dispositivo CoreML: {e}")
else:
    print("⚠️ Pulando operações do dispositivo pois CoreML não está disponível")

## Configuração do Backend CoreML

In [ ]:
if coreml_available:
    try:
        # Criar configuração do backend CoreML
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Configuração do backend: {config}")
        
        # Verificar e modificar valores de configuração
        print(f"📊 Habilitar cache: {config.enable_caching}")
        print(f"🗂️ Tamanho máximo do cache: {config.max_cache_size}")
        print(f"📈 Habilitar profiling: {config.enable_profiling}")
        print(f"🔄 Fallback automático: {config.auto_fallback}")
        
        # Modificar configuração
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Configuração atualizada: {config}")
        
        # Criar backend CoreML
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 Backend CoreML: {backend}")
        print(f"✅ Backend disponível: {backend.is_available()}")
        
        # Obter estatísticas do backend
        stats = backend.get_stats()
        print(f"📊 Estatísticas do backend: {stats}")
        print(f"   Operações totais: {stats.total_operations}")
        print(f"   Acertos de cache: {stats.cache_hits}")
        print(f"   Erros de cache: {stats.cache_misses}")
        print(f"   Operações de fallback: {stats.fallback_operations}")
        print(f"   Taxa de acerto de cache: {stats.cache_hit_rate():.2%}")
        print(f"   Taxa de fallback: {stats.fallback_rate():.2%}")
        print(f"   Tempo médio de execução: {stats.average_execution_time_ms:.2f}ms")
        
        # Limpeza do cache
        backend.cleanup_cache()
        print("\n🧹 Cache do backend limpo")
        
    except Exception as e:
        print(f"❌ Erro na operação do backend CoreML: {e}")
else:
    print("⚠️ Pulando operações do backend pois CoreML não está disponível")

## Operações Básicas de Tensor (CPU)

Para comparar com CoreML, vamos primeiro realizar operações básicas na CPU.

In [ ]:
try:
    # Criação e operações básicas de tensor
    print("🧮 Operações básicas de tensor (CPU)")
    
    # Criar tensores a partir de arrays NumPy (interface simplificada)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Forma da matriz A: {data_a.shape}")
    print(f"📐 Forma da matriz B: {data_b.shape}")
    
    # Multiplicação de matrizes com NumPy (para comparação)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ Forma do resultado matmul NumPy: {numpy_result.shape}")
    print(f"📊 Resultado (primeiros elementos): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 Operações CPU completadas")
    
except Exception as e:
    print(f"❌ Erro na operação de tensor: {e}")

## Simulação de Comparação de Performance

In [ ]:
import time

def benchmark_matrix_operations():
    """Comparar performance com diferentes tamanhos de matriz"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Comparação de performance:")
    print("Tamanho\t\tTempo CPU (ms)\tCoreML Esperado (ms)")
    print("-" * 58)
    
    for size in sizes:
        # Medir tempo de execução CPU
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Tempo CoreML esperado (hipotético)
        # Na implementação real, usar medições reais do backend CoreML
        expected_coreml_time = cpu_time * 0.6  # Suposição: CoreML é 40% mais rápido
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Nota: Os tempos CoreML são hipotéticos. Valores reais dependem da implementação específica.")

## Simulação de Seleção de Dispositivo

In [ ]:
def simulate_device_selection():
    """Simular seleção inteligente de dispositivo"""
    
    operations = [
        ("Multiplicação de matriz pequena", (16, 16), "CPU"),
        ("Multiplicação de matriz média", (128, 128), "Metal GPU"),
        ("Multiplicação de matriz grande", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("Função de ativação", (32, 64, 128, 128), "Metal GPU"),
        ("Convolução pequena", (1, 3, 32, 32), "CPU"),
        ("Convolução grande", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("Operações de números complexos", (128, 128), "Metal GPU"),  # CoreML não suportado
        ("Distribuição estatística", (1000,), "CPU"),  # CoreML não suportado
    ]
    
    print("🎯 Simulação de seleção inteligente de dispositivo:")
    print("Operação\t\t\tForma do Tensor\t\tDispositivo Selecionado")
    print("-" * 78)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<31}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Lógica de seleção:")
    print("  • Operações pequenas: CPU (evitar overhead)")
    print("  • Operações médias: Metal GPU (balanceado)")
    print("  • Operações grandes: CoreML (otimizado)")
    print("  • Operações não suportadas: fallback GPU/CPU")

simulate_device_selection()

## Exemplo Prático: Camada Simples de Rede Neural

In [ ]:
def simulate_neural_network_layer():
    """Simular camada de rede neural"""
    
    print("🧠 Simulação de camada de rede neural:")
    
    # Tamanho do lote e configuração da camada
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 classes
    
    print(f"📊 Tamanho do lote: {batch_size}")
    print(f"🔢 Características de entrada: {input_features}")
    print(f"🧮 Características ocultas: {hidden_features}")
    print(f"🎯 Características de saída: {output_features}")
    
    # Simulação de forward pass
    steps = [
        ("Entrada → Oculta", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("Ativação ReLU", f"({batch_size}, {hidden_features})", "Metal"),
        ("Oculta → Saída", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Simulação de forward pass:")
    total_time = 0
    
    for step, shape, device in steps:
        # Tempo de execução virtual (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Tempo total do forward pass: {total_time:.2f}ms")
    print(f"🚀 Throughput estimado: {1000/total_time:.0f} lotes/segundo")

simulate_neural_network_layer()

## Resumo e Próximos Passos

In [ ]:
print("📋 Resumo da Integração CoreML do RusTorch:")
print()
print("✅ Itens completados:")
print("  • Configuração do ambiente Jupyter")
print("  • Criação do kernel Rust e bindings Python")
print("  • Verificação de disponibilidade CoreML")
print("  • Gerenciamento de dispositivos e configuração")
print("  • Estatísticas e profiling do backend")
print("  • Seleção inteligente de dispositivo")
print()
print("🚧 Desenvolvimento futuro:")
print("  • Implementação real de operações CoreML")
print("  • Benchmarking de performance")
print("  • Mais funções de ativação e tipos de camadas")
print("  • Melhorias no tratamento de erros")
print("  • Otimização de memória")
print()
print("🎯 Próximos passos recomendados:")
print("  1. Carregar e testar modelos CoreML reais")
print("  2. Comparar performance Metal e CoreML")
print("  3. Testar com workflows de deep learning reais")
print("  4. Avaliar em ambiente de produção")

if coreml_available:
    print("\n🎉 Parabéns! CoreML está disponível e todas as funcionalidades podem ser testadas.")
else:
    print("\n⚠️ CoreML não está disponível, mas as funcionalidades básicas estão funcionando.")
    print("   Recomendamos compilar com funcionalidades CoreML habilitadas no macOS.")